In [ ]:
import hydra
import torch
from tqdm.notebook import tqdm
import time
from collections import defaultdict
from lib.utils.config import load_config
from lib.optimizer.framework import NeuralOptimizer, NeuralOptimizer2
from lib.data.loader import load_intrinsics
from lib.data.loader import load_intrinsics
from lib.rasterizer import Rasterizer
from lib.renderer.renderer import Renderer
from lib.renderer.camera import Camera
from lib.tracker.timer import TimeTracker
from lib.utils.progress import reset_progress, close_progress


def path_to_abblation(path):
    return "_".join(path.split("/")[-3].split("_")[1:])


def eval_iterations(optimizer, datamodule, backward: bool=False, strategy: str="forward-mode"):
    tracker = TimeTracker()
    optimizer.max_iters = 1
    optimizer.max_optims = 1
    optimizer.optimizer.strategy=strategy

    # loop
    residuals = []
    for batch in datamodule.val_dataloader():
        batch = optimizer.transfer_batch_to_device(batch, "cuda", 0)
        tracker.start("step")
        out = optimizer(batch)
        if backward:
            out["params"] = batch["init_params"]
            loss_info = optimizer.compute_loss(batch=batch, out=out)
            loss_info["loss"].backward()
        tracker.stop("step")
        residuals.append(out["optim_masks"][0].sum())
    return tracker, sum(residuals) / len(residuals)


def load_flame_renderer(scale):
    # instanciate similar to training
    cfg = load_config("train", ["data=debug"])
    K = load_intrinsics(data_dir=cfg.data.intrinsics_dir, return_tensor="pt")
    camera = Camera(
        K=K,
        width=cfg.data.width,
        height=cfg.data.height,
        near=cfg.data.near,
        far=cfg.data.far,
        scale=scale,
    )
    rasterizer = Rasterizer(width=camera.width, height=camera.height)
    renderer = Renderer(rasterizer=rasterizer, camera=camera)
    flame = hydra.utils.instantiate(cfg.model)
    return flame, renderer


def load_neural_optimizer(flame, renderer, path):
    cfg = load_config("train", ["data=debug"])
    correspondence = hydra.utils.instantiate(cfg.correspondence)
    weighting = hydra.utils.instantiate(cfg.weighting)
    residuals = hydra.utils.instantiate(cfg.residuals)
    regularize = hydra.utils.instantiate(cfg.regularize)
    neural_optimizer = NeuralOptimizer.load_from_checkpoint(
        path,
        renderer=renderer,
        flame=flame,
        correspondence=correspondence,
        regularize=regularize,
        residuals=residuals,
        weighting=weighting,
    )
    return neural_optimizer


# setup the datamodule
def load_datamodule(renderer, start_frame, end_frame):
    cfg = load_config("train", ["data=debug"])
    datamodule = hydra.utils.instantiate(
        cfg.data,
        renderer=renderer,
        val_dataset=dict(
            start_frame=start_frame,
            end_frame=end_frame,
        ),
    )
    datamodule.setup("fit")
    return datamodule

In [ ]:
# settings
start_frame = 10
end_frame = 12

# checkpoints
ours = "/home/borth/GuidedResearch/logs/2024-10-06/06-55-55_abblation_ours_ckpt/checkpoints/last.ckpt"

# loadings
times = defaultdict(dict)
residuals = defaultdict(dict)

for scale in tqdm(range(1, 17)):
    flame, renderer = load_flame_renderer(scale)
    datamodule = load_datamodule(renderer, start_frame, end_frame)
    optimizer = load_neural_optimizer(flame, renderer, ours)
    tracker, residual = eval_iterations(optimizer, datamodule, backward=False, strategy="forward-mode")
    residuals[scale]["fwAD-fw"] = residual
    times[scale]["fwAD-fw"] = tracker.compute_statistics()["step"]["median"]
    tracker, residual = eval_iterations(optimizer, datamodule, backward=True, strategy="forward-mode")
    residuals[scale]["fwAD-fw"] = residual
    times[scale]["fwAD-fw+bw"] = tracker.compute_statistics()["step"]["median"]
    tracker, residual = eval_iterations(optimizer, datamodule, backward=False, strategy="reverse-mode")
    residuals[scale]["fwAD-fw"] = residual
    times[scale]["revAD-fw"] = tracker.compute_statistics()["step"]["median"]
    tracker, residual = eval_iterations(optimizer, datamodule, backward=True, strategy="reverse-mode")
    residuals[scale]["fwAD-fw"] = residual
    times[scale]["revAD-fw+bw"] = tracker.compute_statistics()["step"]["median"]
    
print(times)
print(residuals)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

reverse_no_grad = {
    "method": "reverse_no_grad",
    "16x": 25.015,
    "8x": 82.966,
    "4x": 384.760,
    "2x": 0.0,
    "1x": 0.0,
}
reverse_grad = {
    "method": "reverse_grad",
    "16x": 27.698,
    "8x": 82.966,
    "4x": 477.149,
    "2x": 0.0,
    "1x": 0.0,
}
reverse_diff = {
    "method": "reverse_diff",
    "16x": 166.238,
    "8x": 0.0,
    "4x": 0.0,
    "2x": 0.0,
    "1x": 0.0,
}
forward_no_grad = {
    "method": "forward_no_grad",
    "16x": 11.612,
    "8x": 11.916,
    "4x": 14.858,
    "2x": 27.307,
    "1x": 75.879,
}
forward_grad = {
    "method": "forward_grad",
    "16x": 12.409,
    "8x": 12.446,
    "4x": 15.419,
    "2x": 27.481,
    "1x": 76.292,
}
forward_diff = {
    "method": "forward_diff",
    "16x": 45.590,
    "8x": 46.995,
    "4x": 54.543,
    "2x": 91.013,
    "1x": 243.967,
}
df = pd.DataFrame(
    [
        forward_no_grad,
        forward_grad,
        forward_diff,
        reverse_no_grad,
        reverse_grad,
        reverse_diff,
    ]
)

In [ ]:
# Extracting method names and the x multipliers
methods = df["method"]
x_multipliers = ["16x", "8x", "4x", "2x", "1x"]
# Extracting the values for each multiplier
values = df[x_multipliers].values
# Set up the color map
colors = plt.get_cmap("Set2", len(df["method"]))
# Set up the figure and axes
fig, ax = plt.subplots(figsize=(8, 5))
# Number of bars per group
n_bars = len(df["method"])
# Set the width of the bars
bar_width = 0.15
# Set the spacing between groups
group_spacing = 0.2  # Adjust this value for more or less space between groups
# Set the positions of the bars with added spacing
bar_positions = np.arange(len(df.columns) - 1) * (n_bars * bar_width + group_spacing)
# Plotting each method as a separate bar
for i, (method, color) in enumerate(zip(df["method"], colors.colors)):
    pos = bar_positions + i * bar_width
    ax.bar(pos, values[i], bar_width, label=method, color=color)
# Adding x-ticks
ax.set_xticks(bar_positions + bar_width * (n_bars - 1) / 2)
ax.set_xticklabels(["440 / (67x120)", "1794 / (135x240)", "7138 / (270x480)", "28636 / (540x960)", "115398 / (1080x1920)"])
# Adding labels and title
ax.set_xlabel("Residuals (count) / Resolution (HxW)")
ax.set_ylabel("Time (ms)")
ax.set_title("Jacobian Performance Comparison by Method")
# Adding a red line at 30ms
ax.axhline(y=30, color="red", linestyle="--")
# Adding a legend
ax.legend(loc="upper left")
# Adding grid for better readability
ax.grid(True, linestyle="--", alpha=0.7)
# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Extracting method names and x multipliers
df = df.replace(0.0, np.nan)
methods = df["method"]
x_multipliers = ["16x", "8x", "4x", "2x", "1x"]

# Extracting the values for each multiplier
values = df[x_multipliers].values

# Set up the color map
colors = plt.get_cmap("Set2", len(df["method"]))

# Set up the figure and axes
fig, ax = plt.subplots(figsize=(8, 5))

# Plotting each method as a line
for i, (method, color) in enumerate(zip(df["method"], colors.colors)):
    ax.plot(x_multipliers, values[i], label=method, color=color, marker='o', linewidth=2)

# Adding x-ticks (custom labels)
ax.set_xticks(x_multipliers)
ax.set_xticklabels(["440 / (67x120)", "1794 / (135x240)", "7138 / (270x480)", "28636 / (540x960)", "115398 / (1080x1920)"])

# Adding labels and title
ax.set_xlabel("Residuals (count) / Resolution (HxW)")
ax.set_ylabel("Time (ms)")
ax.set_title("Jacobian Performance Comparison by Method")

# Adding a red line at 30ms
ax.axhline(y=30, color="red", linestyle="--")

# Adding a legend
ax.legend(loc="upper left")

# Show the plot
ax.set_ylim(bottom=0)  # Set minimum y-axis limit to 0
ax.set_xlim(left=-0.2)  # Set minimum y-axis limit to 0
ax.yaxis.grid(True)

# Adding grid for better readability
ax.grid(True, linestyle="--", alpha=0.7)

# Show the plot
plt.tight_layout()
plt.show()


In [11]:
cg = 1.283
ce = 89

x = (cg * 89 - (8 * 1.7) ) / 81
(x * 81 + 1.0 * 5 + 1.0 + 30 + 1.3 * 10) / (81+5+30+10)

1.1871984126984125